In [1]:
import numpy as np
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/eeg-brainwave-dataset-feeling-emotions/emotions.csv


In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization,LSTM


In [3]:
df = pd.read_csv('/kaggle/input/eeg-brainwave-dataset-feeling-emotions/emotions.csv')

In [4]:
df.head()

,# mean_0_a,mean_1_a,mean_2_a,mean_3_a,mean_4_a,mean_d_0_a,mean_d_1_a,mean_d_2_a,mean_d_3_a,mean_d_4_a,...,fft_741_b,fft_742_b,fft_743_b,fft_744_b,fft_745_b,fft_746_b,fft_747_b,fft_748_b,fft_749_b,label
0,4.62,30.3,-356.0,15.6,26.3,1.070,0.411,-15.70,2.06,3.15,...,23.5,20.3,20.3,23.5,-215.0,280.00,-162.00,-162.00,280.00,NEGATIVE
1,28.80,33.1,32.0,25.8,22.8,6.550,1.680,2.88,3.83,-4.82,...,-23.3,-21.8,-21.8,-23.3,182.0,2.57,-31.60,-31.60,2.57,NEUTRAL
2,8.90,29.4,-416.0,16.7,23.7,79.900,3.360,90.20,89.90,2.03,...,462.0,-233.0,-233.0,462.0,-267.0,281.00,-148.00,-148.00,281.00,POSITIVE
3,14.90,31.6,-143.0,19.8,24.3,-0.584,-0.284,8.82,2.30,-1.97,...,299.0,-243.0,-243.0,299.0,132.0,-12.40,9.53,9.53,-12.40,POSITIVE
4,28.30,31.3,45.2,27.3,24.5,34.800,-5.790,3.06,41.40,5.52,...,12.0,38.1,38.1,12.0,119.0,-17.60,23.90,23.90,-17.60,NEUTRAL


In [5]:
df.shape

(2132, 2549)

## PRE-PROCESSING

In [6]:

x = df.iloc[:, :-1].values  
y = df.iloc[:, -1].values   


In [7]:
# Encoding labels (Converting text labels to numbers)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

#Normalizing EEG feature values
scaler = StandardScaler()
x = scaler.fit_transform(x)


In [8]:
x = np.expand_dims(x, axis=2)  # Shape: (2132, 2548, 1)


In [9]:
#Splitting into train and test
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=42)

##  Using CNN

In [10]:
model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(x.shape[1], 1)),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),

    Conv1D(filters=128, kernel_size=3, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(len(np.unique(y)), activation='softmax')  # Output layer for classification
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [12]:
history = model.fit(xtrain, ytrain, epochs=30, batch_size=32, validation_data=(xtest, ytest))


Epoch 1/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 12s 109ms/step - accuracy: 0.7285 - loss: 6.8676 - val_accuracy: 0.6159 - val_loss: 2.6765
Epoch 2/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.8713 - loss: 3.7082 - val_accuracy: 0.4286 - val_loss: 12.4674
Epoch 3/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9017 - loss: 1.8357 - val_accuracy: 0.4450 - val_loss: 18.2283
Epoch 4/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9386 - loss: 0.9429 - val_accuracy: 0.4871 - val_loss: 14.6966
Epoch 5/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9487 - loss: 0.5849 - val_accuracy: 0.8056 - val_loss: 3.1209
Epoch 6/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9616 - loss: 0.7467 - val_accuracy: 0.8478 - val_loss: 2.6010
Epoch 7/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9521 - loss: 0.2889 - val_accuracy: 0.9368 - val_loss: 0.8786
Epoch 8/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9655 - loss: 0.4909 - val_accuracy: 0.934

In [13]:
test_loss, test_acc = model.evaluate(xtest, ytest)
print(f"Test Accuracy: {test_acc:.4f}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9738 - loss: 0.8036     
Test Accuracy: 0.9742


## Using ANN

In [14]:
model1 = Sequential([
    Dense(512, activation='relu', input_shape=(x.shape[1],)),  
    BatchNormalization(),
    Dropout(0.3),

    Dense(256, activation='relu'),  # Hidden layer 1
    BatchNormalization(),
    Dropout(0.3),

    Dense(128, activation='relu'),  # Hidden layer 2
    Dropout(0.3),

    Dense(len(np.unique(y)), activation='softmax')  # Output layer for classification
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
model1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [16]:
model1.fit(xtrain, ytrain, epochs=30, batch_size=32, validation_data=(xtest, ytest))


Epoch 1/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 8s 75ms/step - accuracy: 0.7941 - loss: 0.5498 - val_accuracy: 0.9157 - val_loss: 0.1854
Epoch 2/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8956 - loss: 0.2944 - val_accuracy: 0.9438 - val_loss: 0.1745
Epoch 3/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9191 - loss: 0.2425 - val_accuracy: 0.9391 - val_loss: 0.1555
Epoch 4/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9309 - loss: 0.1844 - val_accuracy: 0.9485 - val_loss: 0.1404
Epoch 5/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9335 - loss: 0.1547 - val_accuracy: 0.9672 - val_loss: 0.1174
Epoch 6/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9471 - loss: 0.1329 - val_accuracy: 0.9696 - val_loss: 0.1095
Epoch 7/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9620 - loss: 0.1185 - val_accuracy: 0.9578 - val_loss: 0.1256
Epoch 8/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9761 - loss: 0.0703 - val_accuracy: 0.9625 - val_loss

In [17]:
test_loss, test_acc = model1.evaluate(xtest, ytest)
print(f"Test Accuracy: {test_acc:.4f}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9790 - loss: 0.0524 
Test Accuracy: 0.9766


##  Using LSTM

In [18]:
model2 = Sequential([
    LSTM(128, return_sequences=True, input_shape=(x.shape[1], 1)), 
    BatchNormalization(),
    Dropout(0.3),

    LSTM(64, return_sequences=False),  
    BatchNormalization(),
    Dropout(0.3),

    Dense(64, activation='relu'),
    Dropout(0.3),
    
    Dense(len(np.unique(y)), activation='softmax')  # Output layer for classification
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [19]:
model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [20]:
model2.fit(xtrain, ytrain, epochs=30, batch_size=32, validation_data=(xtest, ytest))

Epoch 1/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 13s 154ms/step - accuracy: 0.6307 - loss: 0.9231 - val_accuracy: 0.3841 - val_loss: 1.0757
Epoch 2/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 8s 149ms/step - accuracy: 0.7926 - loss: 0.5388 - val_accuracy: 0.5363 - val_loss: 1.1176
Epoch 3/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 8s 149ms/step - accuracy: 0.8292 - loss: 0.4743 - val_accuracy: 0.4005 - val_loss: 1.1407
Epoch 4/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 8s 149ms/step - accuracy: 0.7996 - loss: 0.5202 - val_accuracy: 0.5410 - val_loss: 1.1537
Epoch 5/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 8s 150ms/step - accuracy: 0.8425 - loss: 0.4201 - val_accuracy: 0.5948 - val_loss: 0.9900
Epoch 6/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 8s 151ms/step - accuracy: 0.8358 - loss: 0.4323 - val_accuracy: 0.6674 - val_loss: 1.0779
Epoch 7/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 8s 151ms/step - accuracy: 0.8451 - loss: 0.3973 - val_accuracy: 0.5761 - val_loss: 0.7101
Epoch 8/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 8s 152ms/step - accuracy: 0.8199 - loss: 0.4568 - val_accuracy: 0

In [21]:
test_loss, test_acc = model2.evaluate(xtest, ytest)
print(f"Test Accuracy: {test_acc:.4f}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - accuracy: 0.9164 - loss: 0.1994
Test Accuracy: 0.9297
